In [ ]:
from torchvision.datasets import CIFAR10
from torchvision import transforms

train_data = CIFAR10(root='data', train=True, download=True, transform=transforms.ToTensor())

100%|██████████| 170498071/170498071 [01:51<00:00, 1531742.80it/s]


Extracting data\cifar-10-python.tar.gz to data


In [ ]:
import torch
from torch import nn 
class Cifar10_clf:
    def __init__(self):
        super(Cifar10_clf, self).__init__()

        self.conv = nn.Conv2d()
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d()
        
    def forward(self ,x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.conv2d(x)

SyntaxError: incomplete input (3490684431.py, line 1)